挂载

In [ ]:
from google.colab import drive
 
drive.mount('/content/drive')

切换位置

In [ ]:
%cd drive/MyDrive/Project4
# !ls
import sys
sys.path.append('/content/drive/MyDrive/Project4')

import各种类

In [ ]:
import random
import os
import argparse
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import nltk
nltk.download("all-nltk")
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from collections import Counter

from lib.DataLoader import *
from lib.criterion import *
from lib.loss import *
from lib.optimizer import *
from lib.parser import *
from lib.utils import *
from lib.Mytransformer import *
from evaluate import *

In [ ]:
import copy
import warnings
warnings.filterwarnings("ignore")

训练函数

In [ ]:

def make_model(src_vocab, lab_vocab, N = 6, d_model = 512, d_ff = 2048, h = 8, dropout = 0.1):
  c = copy.deepcopy
  multihead = MultiHeadedAttention(h, d_model).to(args.device)
  feedforward = FeedForward(d_model, d_ff, dropout).to(args.device)
  position = PositionalEncoding(d_model, dropout).to(args.device)
  model = Transformer(
    Encoder(EncoderLayer(d_model, c(multihead), c(feedforward), dropout).to(args.device), N).to(args.device),
    Decoder(DecoderLayer(d_model, c(multihead), c(multihead), c(feedforward), dropout).to(args.device), N).to(args.device),
    nn.Sequential(Embeddings(d_model, src_vocab).to(args.device), c(position)),
    nn.Sequential(Embeddings(d_model, lab_vocab).to(args.device), c(position)),
    Generator(d_model, lab_vocab)).to(args.device)
  
  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)
  return model.to(args.device)

def run_epoch(data, model, loss_compute, epoch):
  total_tokens = 0
  total_loss = 0
  for i , batch in enumerate(data):
    out = model(batch.src, batch.lab, batch.src_mask, batch.lab_mask)
    loss = loss_compute(out, batch.lab_y, batch.ntokens)
    total_loss += loss
    total_tokens += batch.ntokens
    if i % 100 == 1:
      print("epoch %d \t batch: %d \t Loss: %f " % (epoch, i - 1, loss / batch.ntokens))
  return total_loss / total_tokens

def train(data, model, criterion, optimizer):
  for epoch in range(1, 1 + args.epochs):
    print('------- Train epoch : ' +  str(epoch) + "--------")
    model.train()
    run_epoch(data.train_data, model, SimpleLossCompute(model.generator, criterion, optimizer), epoch)
    torch.save(model.state_dict(), "model.pkl")
    model.eval()
    loss = run_epoch(data.valid_data, model, SimpleLossCompute(model.generator, criterion, None), epoch)
    print('Evaluate loss: %f' % loss)

读取数据个数设置

In [ ]:
PAD, BOS, EOS, UNK = 'PAD', 'BOS', 'EOS', 'UNK'
g_max_seq1 = float('inf')
g_max_seq2 = float('inf')
g_max_seq3 = float('inf')

参数设置

In [ ]:
args.layers = 2
args.batch_size = 128
args.d_model = 256
args.d_ff = 1024
args.h_num = 8
args.dropout = 0.1
args.epochs = 20

加载数据

In [ ]:
g_max_seq1 = 1500000 
g_max_seq2 = 150000 
Data = DataHandler()
Data.prepare_train(g_max_seq1, g_max_seq2)
args.src_vocab = Data.en_total_words
args.lab_vocab = Data.cn_total_words
print(args.src_vocab)
print(args.lab_vocab)

模型创建

In [ ]:
# args.src_vocab = 131074 # 794 
# args.lab_vocab = 7582 # 830
model = make_model(args.src_vocab, args.lab_vocab, args.layers, args.d_model, args.d_ff, args.h_num, args.dropout)
# print(model)

模型训练和验证

In [ ]:
print(">>>>>>> start train <<<<<<<")
criterion = LabelSmoothing(args.lab_vocab, padding_idx = 0, smoothing = 0.0)
optimizer = NoamOpt(args.d_model, 1, 2000, torch.optim.Adam(model.parameters(), lr = 0, betas = (0.9, 0.98), eps = 1e-9))
train(Data, model, criterion, optimizer)
print("<<<<<<< finished train >>>>>>>")
torch.save(model.state_dict(), "model.pkl")

测试模型

In [ ]:
g_max_seq3 = 100 # float("inf")
Data.prepare_test(g_max_seq3)
model.load_state_dict(torch.load("model.pkl"))
evaluate(Data, model)